In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import importlib
import sys
import pytz
import datetime
from dateutil.relativedelta import relativedelta
import importlib

In [59]:
if "../" not in sys.path:
    sys.path.append("../")
import appUtilities as au

In [67]:
# Load in dummy data
df = pd.read_pickle('../data/dummy_data.pkl')

In [76]:
# Get Singapore's timezone
sg_timezone = pytz.timezone('Asia/Singapore')

# Get the current date and time in Singapore's timezone
current_date_sg = datetime.datetime.now(sg_timezone)

pd.to_datetime(current_date_sg.date() + pd.DateOffset(days=1))

Timestamp('2024-03-16 00:00:00')

In [74]:
pd.to_datetime(n_months_agox)

Timestamp('2024-03-15 00:00:00')

In [66]:
df.to_pickle("../data/dummy_data.pkl")

# Recreating main page 

Mainlanding page of last 5 months


| Jan | Feb | March | April | May | Type
| :-- | :-- | :-- | :-- | :-- | --- | 
| $1,000 | $3,430 | $500 | $600 | $5,000 | **Total**
| $500 | $1,000 | $300 | $122 | $1,212 | **Dining**
. . .

Can include sparklines as well in st.dataframe

In [4]:
df.head(1)

,Date,Item,Type,Subtype,Location,Payment Method,Amount,ShopBack?,Notes,month,week_start
0,2023-03-17,Prawn Noodles,Dining,Lunch,Work,PayLah,7.5,No,NaN,2023-03-01,2023-03-13


In [5]:
# Get Singapore's timezone
sg_timezone = pytz.timezone('Asia/Singapore')

# Get the current date and time in Singapore's timezone
# current_date_sg = datetime.now(sg_timezone)
current_date_sg = datetime(2024,2,1 , tzinfo=sg_timezone)

# Subtract 5 months from the current date
four_months_ago = current_date_sg - pd.DateOffset(months=4)
# Set the day of the resulting date to 1 to get the first day of the month
first_day_of_month_4_months_ago = four_months_ago.replace(day=1).date()
print(first_day_of_month_4_months_ago)

2023-10-01


In [6]:
df.loc[df['Date']>=pd.to_datetime(first_day_of_month_4_months_ago)].shape

(364, 11)

In [7]:
# # Get data for the month of october
# month_df = au.get_month(df, month='2023-10-01')
# # Get data for onw week in october
# week_df = au.get_week(df, week='2023-10-23')

In [8]:
# filter entries
tdf = df.loc[df['Date']>= pd.to_datetime(first_day_of_month_4_months_ago)]
# Group by month
tdf = tdf.groupby(['month', 'Type']).Amount.sum()
# Pivot long to wide
tdf = tdf.unstack(level=0)
# Reset the name
tdf.columns.name = None

# Sum values for each column
column_sums = tdf.sum()
column_sums.name = "Total" # Set the name of the series

# Append sums as a new row to the DataFrame
tdf = pd.concat([pd.DataFrame(column_sums).T, tdf]) #Add the column sums so they appear at the top

# Set the index name
tdf.index.name = "Type"

# Change the date columns
tdf.columns =  [pd.to_datetime(col).strftime("%b'%y") for col in tdf.columns.values]

# Fill in missing values
tdf = tdf.fillna(0)

In [9]:
tdf

,Oct'23,Nov'23,Dec'23,Jan'24,Feb'24
Type,,,,,
Total,2134.35,1242.90,2352.45,2272.89,1702.26
Dining,835.62,939.09,943.45,782.85,674.86
Healthcare,14.00,37.40,118.60,47.72,21.00
Leisure,115.00,37.16,347.00,0.00,37.50
Shopping,663.81,229.25,139.90,15.00,253.00
Transport,29.22,0.00,50.00,704.42,0.00
Travel,476.70,0.00,753.50,722.90,715.90


In [10]:
q = tdf.iloc[0]
q

Oct'23    2134.35
Nov'23    1242.90
Dec'23    2352.45
Jan'24    2272.89
Feb'24    1702.26
Name: Total, dtype: float64

In [78]:
importlib.reload(au)

<module 'appUtilities' from '/Users/carelchay/git_projects/personal-spending/notebooks/../appUtilities.py'>

## Styling

In [79]:
tdf = au.create_main_table(df)

In [80]:
tdf

,Dec'23,Jan'24,Feb'24,Mar'24
Type,,,,
Dining,$793.52,"$1,042.18",$885.49,$604.48
Healthcare,$0.00,$0.00,$0.00,$93.79
Leisure,$19.23,$58.87,$32.41,$35.08
Shopping,$436.92,"$1,208.33",$240.48,$742.04
Transport,$37.54,$297.60,$41.78,$0.00
Total,"$1,287.21","$2,606.98","$1,200.16","$1,475.39"


# Pie Chart

Generate a pie chart at the `Type` level for the given period


In [86]:
tdf = df.loc[(df.month>= au.get_current_month())
            & (df.Date<au.get_endoff_day())]

In [90]:
sum_df = tdf.groupby('Type').agg(
    total=pd.NamedAgg(column='Amount', aggfunc='sum')
).reset_index().sort_values(by='Type')
sum_df

,Type,total
0,Dining,565.07
1,Healthcare,93.79
2,Leisure,25.23
3,Shopping,639.63


In [111]:
def pie_chart_most_recent(df:pd.DataFrame):
    """
    Makes a pie chart of spending for the most recent month
    """
    tdf = df.loc[(df.month>= au.get_current_month())
            & (df.Date<au.get_endoff_day())]
    tdf = tdf.groupby('Type').agg(
        total=pd.NamedAgg(column='Amount', aggfunc='sum')
    ).reset_index().sort_values(by='Type')

    # Create a donut chart using Plotly Graph Objects
    fig = go.Figure()
    values = sum_df.total.values
    labels = sum_df.Type.values
    fig.add_trace(go.Pie(
        labels=labels,
        values=values,
        hole=0.5,  # Set the hole parameter to create a donut chart
        marker=dict(colors=['red', 'green', 'blue', 'orange'])  # You can customize colors if needed
    ))
    total_spending = sum(values)
    fig.update_layout(
        # title='Donut Chart Example',
        annotations=[dict(text=f'${total_spending:,.0f}', x=0.5, y=0.5, font_size=20, showarrow=False)]
        , legend= dict(x = 0, y=-0.1, orientation='h')
    )

    return fig

In [112]:
f = pie_chart_most_recent(df)

In [113]:
f.show()

In [8]:
# Create a donut chart using Plotly Graph Objects
fig = go.Figure()
values = sum_df.total.values
labels = sum_df.Type.values
fig.add_trace(go.Pie(
    labels=labels,
    values=values,
    hole=0.5,  # Set the hole parameter to create a donut chart
    marker=dict(colors=['red', 'green', 'blue', 'orange'])  # You can customize colors if needed
))
total_spending = sum(values)
fig.update_layout(
    # title='Donut Chart Example',
    annotations=[dict(text=f'${total_spending:,.0f}', x=0.5, y=0.5, font_size=20, showarrow=False)]
    # , legend= dict(x = 0.5, y=-0.1, orientation='h')
)

fig.show()

# Monthly / weekly Comparison

Comparing spending of main types over the last `n` months / weeks

In [9]:
# How many months to take
n_months = 4
# Get the first day of the current month
current_month = datetime.date.today().replace(day=1)
# Get the first day of the month for the last n months (this month included)
last_n_months = list(map(lambda x: (current_month-relativedelta(months=x)), range(0, n_months)))
# Get the relevant entries (entries in the last n_months)
n_months_df=df.loc[df.month.isin(last_n_months)]

In [10]:
# Aggregate the spending at the type-month level
agg_df=n_months_df.groupby(['Type', 'month']).agg(
    total = pd.NamedAgg(column='Amount', aggfunc='sum')
).reset_index()

agg_df.dtypes

Type             object
month    datetime64[ns]
total           float64
dtype: object

In [11]:
# Cycle colors
color_mapper = ['red', 'green', 'blue', 'orange']
# Get the types
types = sorted(agg_df.Type.unique())
# Plotting the figure
fig = go.Figure()
# Track color cycle
color_id = 0
for group, tdf in agg_df.groupby("Type"):
    color = color_mapper[color_id%len(color_mapper)]
    fig.add_trace(
        go.Scatter(
            x = tdf.month.values
            , y= tdf.total.values
            , mode='lines+markers'
            , name=group
            , line=dict(color=color, width=2)
            , marker=dict(symbol='diamond', size=8)

        )
    )
    color_id+=1

# Customize layout
fig.update_layout(
    title='Prices of Items Over the Past Few Months',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Price'),
    # legend=dict(x=0.02, y=0.98),
    margin=dict(l=50, r=50, t=50, b=50)
)    

In [12]:
fig.show()

# Table View

To recreate the table view for the monthly / weekly breakdown for the last n (fixed at n=4) months. 

Instead of sparklines will use streamlit's inbuilt [`column_config`](https://docs.streamlit.io/library/api-reference/data/st.column_config/st.column_config.barchartcolumn) utility to mimic the chart behaviour. 

This requires creating an additional column where each element is a list of amount spend for that `Type` (row) for the last n months. 

In [47]:
# How many months to take
n_months = 4
# What columns to use
columns = ['month', 'Type', 'Subtype', 'Amount']
# Get the first day of the current month
current_month = datetime.date.today().replace(day=1)
# Get the first day of the month for the last n months (this month included)
last_n_months = list(map(lambda x: (current_month-relativedelta(months=x)), range(0, n_months)))
# Get the relevant entries (entries in the last n_months)
n_months_df=df.loc[df.month.isin(last_n_months), columns].copy()

In [59]:
# Perform Aggregation
tdf = n_months_df.groupby(['month', 'Type']).agg(
    total = pd.NamedAgg(column='Amount', aggfunc='sum')
)

# Need to do this to always ensure the months are in ascending order, and the order of appearance of the types are standardized
tdf = tdf.sort_index(level=0, ascending=True)#.unstack(level=0)
# Reformat the months. Need to change it back to a column first otherwise it will be an 'object' when used as an index. Can try changing it to a dateTimeIndex in the future
tdf = tdf.reset_index("month")
# Re-format to show only the month and year (both abbreviated)
tdf.month = tdf.month.dt.strftime("%b \'%y")
# Add the correctly formatted month column back as an index
tdf=tdf.set_index(['month'], append=True)
# Pivot
tdf = tdf.unstack(-1)

In [60]:
# Collapse the multi column-index
tdf.columns =  [col[1] for col in tdf.columns.values]
# Crate list of values
tdf['temp']=tdf[list(tdf.columns)].values.tolist()

In [61]:
tdf

,Aug '23,Nov '23,Oct '23,Sep '23,temp
Type,,,,,
Dining,637.36,817.96,835.62,555.09,"[637.36, 817.9599999999999, 835.62, 555.09]"
Healthcare,74.75,37.40,14.00,62.25,"[74.75, 37.4, 14.0, 62.25]"
Leisure,30.27,37.16,115.00,23.00,"[30.27, 37.16, 115.0, 23.0]"
Shopping,303.60,213.75,663.81,598.66,"[303.6, 213.75, 663.81, 598.66]"
Transport,198.77,NaN,29.22,195.99,"[198.77, nan, 29.22, 195.99]"
Travel,NaN,NaN,476.70,NaN,"[nan, nan, 476.7, nan]"


# Calendar Plot

[Plotly API reference](https://plotly.com/python/heatmaps/#:~:text=12-,Heatmap%20with%20Datetime%20Axis,-import%20plotly.graph_objects)

[Existing package](https://github.com/brunorosilva/plotly-calplot)